In [ ]:
##mxwwnote---mlo和cc判断使用，仅需要调用f_mlo函数输入路径即可，可自行修改输入方式和输出，已加入注释。
import os
import cv2


def f_mlo(image):
  
    # image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)####需要留意读入得为灰度图

    # 裁剪图像：去除左右上下各5像素
    # cropped_image = image[5:-5, 5:-5]
    cropped_image = image

    height, width = cropped_image.shape[:2]
    left_half = cropped_image[:, :width // 2]
    right_half = cropped_image[:, width // 2:]
    left_sum = cv2.sumElems(left_half)[0]
    right_sum = cv2.sumElems(right_half)[0]
    left_true = (left_sum > right_sum)
    # 分别获取图像的上、下、左、右边界像素
    top_pixels = cropped_image[0, :]
    bottom_pixels = cropped_image[height - 1, :]
    left_pixels = cropped_image[:, 0]
    right_pixels = cropped_image[:, width - 1]

   
    top_nonzero = cv2.countNonZero(top_pixels)
    bottom_nonzero = cv2.countNonZero(bottom_pixels)
    left_nonzero = cv2.countNonZero(left_pixels)
    right_nonzero = cv2.countNonZero(right_pixels)

    
    top_length = width
    bottom_length = width
    left_length = height
    right_length = height

    # 检查非黑像素数量是否超过边界长度的 1/8
    threshold = 1 / 8

    count = 0

    if top_nonzero > top_length * threshold:
        count += 1

    if bottom_nonzero > bottom_length * threshold:
        count += 1

    if left_nonzero > left_length * threshold:
        count += 1

    if right_nonzero > right_length * threshold:
        count += 1

##字符输出，
    if count >= 2 and left_true:
        return "lmlo"
    elif count >= 2 and not left_true:
        return "rmlo"
    elif count < 2 and left_true:
        return "lcc"
    elif count < 2 and not left_true:
        return "rcc"

file_path=r'/data/sjwlab/breast_data/onePdata/two_all_data/images/train'

for file in os.listdir(file_path):
    imagefilepath=os.path.join(file_path,file)
    rememberflag=''
    for oneimage in os.listdir(imagefilepath):
        imagepath=os.path.join(imagefilepath,oneimage)
        im=cv2.imread(imagepath,0)
        flag=f_mlo(im)[1:]
        if flag==rememberflag:
            if rememberflag=='mlo':
                flag='cc'
            else:
                flag='mlo'
        rememberflag=flag
        newimagepath=os.path.join(imagefilepath,flag+'.png')
        os.rename(imagepath,newimagepath)



In [ ]:
import os
import cv2
import numpy as np

def find_largest_contour_and_resize(thresh):
    # _, thresh = cv2.threshold(gray, 90, 255, cv2.THRESH_BINARY_INV)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    max_area = 0
    largest_contour = None

    for contour in contours:
        area = cv2.contourArea(contour)
        if area > max_area:
            max_area = area
            largest_contour = contour

    x, y, w, h = cv2.boundingRect(largest_contour)

    new_img = np.zeros((h, w), dtype=np.uint8)

    # 将最大轮廓内容复制到新图像中
    # 注意：这里直接复制可能会导致位置偏移，因为需要考虑原轮廓在原图中的位置
    # 使用ROI（感兴趣区域）方法更准确地提取并放置轮廓
    roi = thresh[y:y + h, x:x + w]
    new_img[:h, :w] = roi  # 这里简单赋值，实际应根据roi调整大小以适应新图大小，特别是当新图大小与roi不完全匹配时

    return new_img

in_path=r'/data/sjwlab/breast_data/onePdata/two_all_data/test'

for file1 in os.listdir(in_path):
    p1=os.path.join(in_path,file1)
    for file2 in os.listdir(p1):
        p2=os.path.join(p1,file2)
        im=cv2.imread(p2,0)
        croim=find_largest_contour_and_resize(im)
        cv2.imwrite(p2,croim)


In [4]:
import os
import cv2
import numpy as np
from concurrent.futures import ThreadPoolExecutor

def find_largest_contour_and_resize(thresh):
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_area = 0
    largest_contour = None
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > max_area:
            max_area = area
            largest_contour = contour
    if largest_contour is not None:
        x, y, w, h = cv2.boundingRect(largest_contour)
        new_img = np.zeros((h, w), dtype=np.uint8)
        roi = thresh[y:y + h, x:x + w]
        new_img[:h, :w] = roi
        return new_img
    else:
        return None

def process_image(p2):
    im = cv2.imread(p2, 0)
    croim = find_largest_contour_and_resize(im)
    if croim is not None:
        cv2.imwrite(p2, croim)

in_path = r'/data/sjwlab/breast_data/onePdata/two_all_data/images/val'
image_paths = []

for file1 in os.listdir(in_path):
    p1 = os.path.join(in_path, file1)
    for file2 in os.listdir(p1):
        p2 = os.path.join(p1, file2)
        image_paths.append(p2)

# 创建线程池
with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    # 使用线程池的map函数来分配任务
    executor.map(process_image, image_paths)


In [3]:
# def delate_letter(thresh):
#     # _, thresh = cv2.threshold(gray, 90, 255, cv2.THRESH_BINARY_INV)

#     contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     max_area = 0
#     largest_contour = None

#     for contour in contours:
#         area = cv2.contourArea(contour)
#         if area > max_area:
#             max_area = area
#             largest_contour = contour

#     if largest_contour is not None:
#         x, y, w, h = cv2.boundingRect(largest_contour)

#         # 创建一个与原始二值化图像大小相同的零矩阵，用于黑色背景
#         new_img = np.zeros(thresh.shape, dtype=np.uint8)

#         # 使用ROI（感兴趣区域）方法更准确地提取并放置轮廓
#         roi = thresh[y:y + h, x:x + w]
#         new_img[y:y + h, x:x + w] = roi  # 将轮廓复制到新图像的正确位置

#         return new_img
#     else:
#         # 如果没有找到轮廓，返回一个与thresh大小相同的零矩阵
#         return np.zeros(thresh.shape, dtype=np.uint8)

# im=cv2.imread('/data/sjwlab/breast_data/onePdata/two_all_data/test/cbis_P_00001_LEFT/cc.png',0)
# new=delate_letter(im)
# cv2.imwrite('/data/sjwlab/breast_data/onePdata/two_all_data/mid/mid.png',new)

True